In [1]:
import numpy as np
import numpy.linalg as LA
import torch
comp_device = torch.device("cpu")
from human_body_prior.body_model.body_model import BodyModel
from human_body_prior.tools.omni_tools import copy2cpu as c2c
import json
import time
import open3d as o3d

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from SteepestDescent import SteepestDescent_1
from SteepestDescent import SteepestDescent_2
from SteepestDescent import SteepestDescent_3
from SteepestDescent import SteepestDescent_4

from Orient import Orient


## setup path and body model
bm_path = '../support_data/body_models/smplh/neutral/model.npz'
dmpl_path = '../support_data/body_models/dmpls/neutral/model.npz'
num_betas = 10 # number of body parameters
num_dmpls = 8


C:\Users\swoosh\anaconda3\envs\SMPL_BODY_MODEL\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def matrix_dist(A,B):
    #A and B must be of same shape
    
    L = len(A)
    mag = np.empty((L))
    
    for i in range(L):
        mag[i] = LA.norm(A[i] - B[i])
        
    return mag

In [3]:
#randomly sample SMPL shape distribution
N = 100

betas_store = np.empty((N,10))
#mesh_store = np.empty((N,6890,3))
magnitude_store = np.empty((6890,N))

pose_body = np.zeros((1,63))
pose_body = torch.Tensor(pose_body).to(comp_device) # controls the body

bm = BodyModel(bm_fname=bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=dmpl_path).to(comp_device)

#define average vertices
betas = np.zeros((1,10))
betas = torch.Tensor(betas).to(comp_device) # controls the body shape
body = bm(pose_body = pose_body, betas=betas)
vertices_0 = c2c(body.v[0])

for i in range(N):
    
    #create random betas vector
    betas = 6*np.random.random((1,10))
    betas_store[i,:] = betas
    betas = torch.Tensor(betas).to(comp_device) # controls the body shape
    
    #feed into smpl scripts
    body = bm(pose_body=pose_body, betas=betas)
    
    #Get mesh vertices
    vertices = c2c(body.v[0])
    #mesh_store[i][:][:] = vertices
    
    #calculate magnitude of node movement
    magnitude = matrix_dist(vertices,vertices_0)
    magnitude_store[:,i] = magnitude 
    
    #Save shape parameters and mesh vertices in database.
    

In [4]:
#construct matrix to determine covariance

In [7]:
import pandas as pd

matrix = np.empty((N,10+6890))
matrix[:,0:10] = betas_store
matrix[:,10:10+6890] = np.transpose(magnitude_store)
print(matrix.shape)

(100, 6900)


In [ ]:
df = pd.DataFrame(data = matrix)

corr_matrix = df.corr()
np.savetxt("ShapeValue_Vertices_Correlation_N100.txt", corr_matrix, fmt = '%f')

In [6]:
print(betas)

tensor([[3.8907, 4.2118, 4.6201, 3.3946, 5.7722, 5.2770, 4.9570, 4.6209, 5.5562,
         0.3414]])
